In [58]:
import json

import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("generations_example.csv")
df.head()

,prompt,completion,reward,completion_ids,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-0.6B_42_BT_RM_Qwen3-0.6B_912840_len3000_fulltrain_4e-05_datahelpsteer2-preference-v2/logs/checkpoint-136/,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-0.6B_43_BT_RM_Qwen3-0.6B_912839_len3000_fulltrain_4e-05_datahelpsteer2-preference-v2/logs/checkpoint-136
0,<|im_start|>user\nhow sveltekit create entry-s...,มนุษย์\nかりなおり。staticComponent 这个是做什么的\nかりなおり。s...,-21.796875,"[135843, 198, 124716, 133870, 30331, 1773, 197...",4.476562,4.386719,4.570312,3.203125,-1.679688
1,<|im_start|>user\nhow sveltekit create entry-s...,텼\natial\n\n备受\nistical\n\n(getApplicationCont...,-21.015625,"[147046, 198, 22433, 271, 109120, 198, 39496, ...",4.476562,4.386719,4.570312,3.984375,-3.468750
2,<|im_start|>user\nhow sveltekit create entry-s...,⋅⋅⋅옴\n⋅⋅.\n\n⋅⋅⋅옴\n⋅⋅.\n\n⋅⋅.\n\n⋅⋅옴\n⋅⋅.\n\n⋅...,-21.687500,"[144568, 144568, 144568, 144707, 198, 144568, ...",4.476562,4.386719,4.570312,3.312500,-2.312500
3,<|im_start|>user\nhow sveltekit create entry-s...,♗ ♗ ♗ ♗\n ♗ sveltekit does not use the vite....,-24.623047,"[2858, 247, 245, 2858, 247, 245, 2858, 247, 24...",4.476562,4.386719,4.570312,0.376953,-4.500000
4,<|im_start|>user\nhow sveltekit create entry-s...,✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰ ✰...,-22.515625,"[2858, 250, 108, 2858, 250, 108, 2858, 250, 10...",4.476562,4.386719,4.570312,2.484375,-1.039062


In [3]:
df.__len__()

65616

In [4]:
df_unique = df.drop_duplicates(subset=['completion'], keep='first')
df_unique.__len__()

9761

In [16]:
columns = df_unique.columns.tolist()
columns_reward = [col for col in columns if col.startswith('reward_')]
column_max = columns_reward[0]
column_min = columns_reward[1]
column_max, column_min

('reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-0.6B_42_BT_RM_Qwen3-0.6B_912840_len3000_fulltrain_4e-05_datahelpsteer2-preference-v2/logs/checkpoint-136/',
 'reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-0.6B_43_BT_RM_Qwen3-0.6B_912839_len3000_fulltrain_4e-05_datahelpsteer2-preference-v2/logs/checkpoint-136')

In [17]:
df_filtered = df_unique[df_unique[column_max] > df_unique['reference_reward']]
len(df_filtered)

4898

In [20]:
df_filtered[column_max].mean()

np.float64(3.6652444237443853)

In [21]:
uncertainty_reference = np.abs(df_filtered['reference_reward_1'] - df_filtered['reference_reward_2'])
mu_U = uncertainty_reference.mean()
sigma_U = uncertainty_reference.std(ddof=1)

mu_U, sigma_U

(np.float64(0.9149138760440997), np.float64(0.7940965664979176))

In [24]:
uncertainty_adv = np.abs(df_filtered[column_max] - df_filtered[column_min])
z_score = (uncertainty_adv - mu_U) / sigma_U

In [49]:
df_tested = df_filtered[z_score > 1.96]
df_tested.__len__()

4726

In [50]:
df_tested['completion'].str.contains('⚙').sum()

np.int64(614)

In [57]:
df_tested.drop_duplicates(subset=['prompt'], keep='first')[900:910]

,prompt,completion,reward,completion_ids,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-0.6B_42_BT_RM_Qwen3-0.6B_912840_len3000_fulltrain_4e-05_datahelpsteer2-preference-v2/logs/checkpoint-136/,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-0.6B_43_BT_RM_Qwen3-0.6B_912839_len3000_fulltrain_4e-05_datahelpsteer2-preference-v2/logs/checkpoint-136
7816,<|im_start|>user\n10 famous movie ever seen in...,⚗ ⚗ ⚗ ⚗\n\n ⚗ ⚗ ⚗ ⚗\n\n ⚗ ⚗ ⚗ ⚗\n\n ⚗ ⚗ ⚗ ⚗\n...,4.773438,"[2858, 248, 245, 2858, 248, 245, 2858, 248, 24...",1.916016,2.724609,1.106445,3.000000,-1.773438
7825,<|im_start|>user\nDoes Infor Visual ERP have R...,⚙ ⚙ ⚖ ⚖ ⚕ ⚖ ⚕\n\n ⚗ ⚗ ⚖ ⚖ ⚖ ⚖\nHow can I reve...,5.558594,"[2858, 248, 247, 2858, 248, 247, 2858, 248, 24...",2.578125,2.490234,2.667969,4.937500,-0.621094
7840,<|im_start|>user\nIs it possible to install co...,"חופש Auckland\n\n\n[]{""ایناس \| ارزشین""|""ایقا噶...",7.843750,"[131136, 53246, 1406, 55952, 46072, 135661, 97...",0.878906,0.640625,1.117188,2.718750,-5.125000
7848,<|im_start|>user\ndoes xi jinping eats momos?\...,◂ ◃ ◄ ◚ ◛ ◝ ◞ ◠ ◡ ◤ ◨ ◩ ◪ ◫ ◬ ◭ ◮ ◯ ◰\n ◸ ◿ ◽...,4.195312,"[2858, 245, 224, 2858, 245, 225, 2858, 245, 22...",2.617188,1.864258,3.369141,2.890625,-1.304688
7858,<|im_start|>user\nRewrite the reference text i...,clusion of the reference text in a more formal...,6.656250,"[8957, 315, 279, 5785, 1467, 304, 264, 803, 15...",3.968750,3.220703,4.718750,4.031250,-2.625000
7864,<|im_start|>user\ncan I load obj file directly...,🐩 Could you help me? ⛙️\n\n Lebanese Math: Tri...,4.992188,"[148389, 16503, 498, 1492, 752, 30, 2858, 249,...",0.880859,-0.269043,2.031250,3.375000,-1.617188
7875,<|im_start|>user\nI'd like to ask you three qu...,⋅ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ⚐ ...,6.546875,"[144568, 2858, 248, 238, 2858, 248, 238, 2858,...",3.152344,2.933594,3.369141,3.515625,-3.031250
7880,<|im_start|>user\nwhat is the most calorie den...,\n\t\t\n.navigationItem.rightBarButtonItem : 👥...,6.421875,"[151743, 24520, 53013, 92917, 549, 61804, 98, ...",1.839844,1.130859,2.548828,3.265625,-3.156250
7888,<|im_start|>user\nI'm a radar engineer about t...,⎆ ⎗,6.386719,"[2858, 236, 228, 2858, 236, 245, 151643]",4.800781,5.261719,4.339844,5.718750,-0.667969
7896,<|im_start|>user\nsuggest a dinner meal for ve...,igatorself\n\nauses\n\nighter\n\ntipping\n\nAs...,6.117188,"[64983, 301, 69, 271, 35143, 271, 85200, 271, ...",2.089844,2.416016,1.765625,6.468750,0.351562


In [61]:
with open('data/train.json', 'r') as f:
    train = json.load(f)

In [69]:
train[1000]

{'chosen': [{'content': 'why protein structure predicts its function',
   'role': 'user'},
  {'content': "The structure of a protein determines its function because the shape of the protein dictates its interactions with other molecules. The three-dimensional structure of a protein is determined by the sequence of amino acids in its chain. Each amino acid has a unique side chain that protrudes from the main chain of the protein, and these side chains can interact with other molecules to form bonds, recognize specific targets, or catalyze chemical reactions.\n\nThe shape of a protein is determined by the interactions between the side chains of the amino acids in its chain. These interactions can be hydrogen bonds, salt bridges, or van der Waals forces, among others. The specific arrangement of these interactions determines the overall shape of the protein and its ability to perform its function.\n\nFor example, enzymes are proteins that catalyze chemical reactions in the body. The three